<a href="https://colab.research.google.com/github/cicyfan/spark-fundamentals/blob/master/spark_exericises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Spark Exercises

## Initiate 

In [10]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

In [9]:
import findspark
from pyspark.sql import SparkSession

findspark.init()

spark = SparkSession\
        .builder\
        .appName("HW2_cf1311")\
        .getOrCreate()

sc = spark.sparkContext
sc


ModuleNotFoundError: ignored

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Question 1 - BreadBasket_DMS.csv


In [8]:
#solution 1 with DF operations

from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import substring_index, to_date, asc, desc

df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema","/content/drive/My Drive/data/BreadBasket_DMS.csv")

#filter empty items
df1 = df.filter(df["item"] != "NONE")

#create new df with formatted data and rename columns
df2 = df1.select("item", \
                #format date timestamp to exclude time
                to_date(df["date"], 'yyyy-mm-dd'), \
                #extract hour from timestamp
                substring_index(df["time"], ":", 1).alias("hour")) \
         .toDF("item", "date", "hour")

#df operations to order, group, and count               
df3 = df2.orderBy("item", "date", "hour") \
         .groupBy("item", "date", "hour") \
         .count() 

df3.show()


ModuleNotFoundError: ignored

In [ ]:
# solution 2 using SQL

df2.registerTempTable("bakery")
df4 = spark.sql("SELECT item, date, hour, count(*) as count " +
                       "FROM bakery " +
                       "GROUP BY item, date, hour " +
                       "ORDER BY item, date, hour " 
               )


df4.show()

## Question 2 - Restaurants_in_Durham_County_NC.json

In [ ]:
df = spark.read.json("/content/drive/My Drive/data/Restaurants_in_Durham_County_NC.json")

df1 = df.select("fields.rpt_area_desc")\
        .orderBy("rpt_area_desc")\
        .groupBy("rpt_area_desc")\
        .count()\
        .orderBy(desc('count'))

df1.show()


# Question 3 - populationbycountry19802010millions.csv

In [ ]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import col, asc, desc, lit

df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema","true") \
    .load("/content/drive/My Drive/data/populationbycountry19802010millions.csv")

#cast all columns with numbers to float
cols = df.schema.names
for col_name in cols:
    if col_name != '_c0':
        df = df.withColumn(col_name, col(col_name).cast('float'))

#set iterative variables
length = len(cols)-2
begin = 1981
end = 1981+length

#iterate through columns to extract max percent each year and print them
for x in range(begin, end):
    col1 = col(str(x-1))
    col2 = col(str(x))
    
    df1 = df.withColumn("percent", ((col2-col1)/col1) * 100)\
            .withColumn("year", lit(x))\
            .orderBy(desc("percent")).limit(1)\
            .select("year", df["_c0"].alias("country"), "percent" )
   
    df1.show()

# Question 4 - wordCount romeo-juliet-pg1777.txt

In [ ]:
#solution 1, use flatmap and map then DF operations to aggregate, sort, count, and display the results

import re
from pyspark.sql.functions import asc, desc

#read text file
text = sc.textFile("/content/drive/My Drive/data/romeo-juliet-pg1777.txt")

#1) use regex to replace all unacceptable symbols
#2) split words
#3) assign key value pairs where key is word and value is 1
counts = text.map(lambda lines: re.sub('[^0-9a-zA-Z]', ' ', lines.lower()))\
             .flatMap(lambda word: word.split())\
             .map(lambda word: (word, 1))\

df = counts.toDF()\
           .toDF('word', 'count')\
           .groupBy('word')\
           .count()\
           .orderBy(desc("count"))

df.show()


In [ ]:
#solution 2, instead of DF operations, use reduceByKey then print output

counts1 = counts.reduceByKey(lambda word, word1: word + word1) \
#collect everything in an array
output = counts1.collect()
#print(output)
for (word, count) in output: print("%s: %i" % (word, count))


# Question 5 - Bonus


In [ ]:
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import col, sqrt, abs, desc, asc

#process foreclosure data
fc = spark.read.json("/mnt/c/users/cicyf/data/durham-nc-foreclosure-2006-2016.json")
fc1 = fc.select("recordid", "fields.geocode") 
fc2 = fc1.select("recordid",\
                 fc1["geocode"].getItem(0).alias("lat"),\
                 fc1["geocode"].getItem(1).alias("long"))

#process restaurant data
rest = spark.read.json("/mnt/c/users/cicyf/data/Restaurants_in_Durham_County_NC.json")
rest1 = rest.select("recordid", "fields.status", "fields.rpt_area_desc", "fields.geolocation", "geometry.coordinates")
rest2 = rest1.filter(rest1['status'] == "ACTIVE")\
             .filter(rest1['rpt_area_desc'] == 'Food Service')
rest3 = rest2.select("recordid",\
                     rest2["geolocation"].getItem(0).alias("lat"),\
                     rest2["geolocation"].getItem(1).alias("long"))

#distance formula given in class
r = 0.017
def insideCircle(x1, y1, x2, y2, r):
    return (sqrt((abs(x2-x1)**2) + (abs(y2-y1)**2)) <= r )

spark.udf.register("inCirc", insideCircle, BooleanType())

#cross join
df = rest3.crossJoin(fc2).toDF("restID", "lat1", "long1", "foreID", "lat2", "long2")

df1 = df.withColumn("closure", insideCircle(df.lat1, df.long1, df.lat2, df.long2, r))
df2 = df1.filter(df1.closure == 1 )\
         .groupBy('restID')\
         .count()\
         .orderBy(desc('count'))

df2.show()
